In [1]:
import os
import sys
import glob
import torch
import spacy
import ray
import json
import tqdm
import warnings
from typing import List

from utils import merge_short_sents

In [2]:
ray.init(num_cpus=64)

2020-02-27 13:19:51,118	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-02-27 13:19:51,119	INFO resource_spec.py:216 -- Starting Ray with 71.92 GiB memory available for workers and up to 18.63 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '169.237.10.101',
 'redis_address': '169.237.10.101:22792',
 'object_store_address': '/tmp/ray/session_2020-02-27_13-19-51_118284_31813/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-02-27_13-19-51_118284_31813/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-02-27_13-19-51_118284_31813'}

In [5]:
DATA_PATH = "/home/wuqy1203/Desktop/English-Corpus/RAW/books_txt/"
TEMP_PATH = "book_temp"
STORE_PATH = ""

In [6]:
dir_paths = os.listdir(DATA_PATH)

In [7]:
all_filenames = []

for dir_path in dir_paths:
    dir_path = os.path.join(DATA_PATH, dir_path)
    # we only care about txt files
    filenames = glob.glob(os.path.join(dir_path, "*.txt"))
    all_filenames.extend(filenames)

In [8]:
print(f"total number: {len(all_filenames)}")

total number: 11039


In [7]:
# initialize SpaCy
spacy.prefer_gpu()
# we only need sentencizer
nlp = spacy.load("en", disable=["ner", "tagger", "parser", 
                                "merge_noun_chunks", "merge_entities", "merge_subtokens",
                               "entity_ruler", "textcat", "entity_linker"])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 1212504981

1000000

In [9]:
filename = all_filenames[0]

In [10]:
with open(filename, "r") as f_read:
    data = f_read.read()
    
spacy_doc = nlp(data)
doc_sents = [sent.text for sent in spacy_doc.sents]
doc_sents = merge_short_sents(doc_sents)

In [11]:
print("Processed Example")
print(doc_sents)

Processed Example
['\ufeffA\nBride for\nTom\n\nRuth Ann Nordin\n\nRuth Ann Nordin’s Books\nSpringfield, Nebraska\nA Bride for Tom - Smashwords Edition\nPublished by Ruth Ann Nordin at Smashwords\nCopyright © 2010 by Ruth Ann Nordin\nAll rights reserved.', 'No part of this book may be reproduced or transmitted in any form or by any means, electronic or mechanical, including photocopying, recording, or by any information storage and retrieval system, without permission in writing from the copyright owner.', '\nThis is a work of fiction.', 'Names, characters, places and incidents are either the product of the author’s imagination or are used fictitiously, and any resemblance to any actual persons, living or dead, events, or locales is entirely coincidental.', '\nSmashwords Edition, License Notes:\nThis ebook is licensed for your personal enjoyment only.', 'This ebook may not be re-sold or given away to other people.', 'If you would like to share this book with another person, please purch

In [12]:
def process_document(filename:str) -> List[str]:
    with open(filename, encoding="utf8", errors='ignore') as f_read:
        data = f_read.read()

    spacy_doc = nlp(data)
    doc_sents = [sent.text for sent in spacy_doc.sents]
    doc_sents = merge_short_sents(doc_sents)
    
    return doc_sents
        
@ray.remote
def write_data(filenames:List[str], file_descripts:List[str]=None, output_name=None, sector_id=None):
    ""
    f_write = open(output_name, "w+")
    # write file descriptions

    
    if sector_id == 0:
        pbar = tqdm.tqdm(filenames)
    else:
        pbar = filenames
    
    for i, filename in enumerate(pbar):

        desp = file_descripts[i][0]
        
        try:
            doc_sents = process_document(filename)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            continue
            
        # dump into json
        line = {"id": desp, 
                "sents": doc_sents}
        line = json.dumps(line)
        
        # write into jsonl
        f_write.write(line)
        f_write.write("\n")
    
    f_write.close()
    
    return 0

In [13]:
# define the sector size
total_num_sector = 64

In [14]:
sector_path = os.path.join(STORE_PATH, "SECTOR")
meta_path = os.path.join(STORE_PATH, "META")

# make directories
os.makedirs(sector_path, exist_ok=True)
os.makedirs(meta_path, exist_ok=True)

file_descripts = []

# write descriptions
for sector_id in range(total_num_sector):
    sector_filenames = all_filenames[sector_id::total_num_sector]
    file_ids = ["book_corpus_" + str(i + sector_id * total_num_sector) 
                for i in range(sector_id, len(all_filenames), total_num_sector)]
    file_descripts.extend([(file_id, os.path.join(sector_path, str(sector_id) + ".jsonl"))
                            for file_id in file_ids])

assert len(all_filenames) == len(file_descripts) == len(set([item[0] for item in file_descripts]))

In [ ]:
# Fire Ray Remote
remote_objs = []

for sector_id in range(total_num_sector):
    sector_filenames = all_filenames[sector_id::total_num_sector]
    
    sector_file_ids = ["book_corpus_" + str(i + sector_id * total_num_sector) 
                for i in range(sector_id, len(all_filenames), total_num_sector)]
    
    sector_file_descripts = [(file_id, os.path.join(sector_path, str(sector_id) + ".jsonl"))
                            for file_id in sector_file_ids]
    
    obj_id = write_data.remote(sector_filenames, 
                               sector_file_descripts, 
                               os.path.join(sector_path, f"{sector_id}.jsonl"), 
                               sector_id)
    remote_objs.append(obj_id)
        
for sector_id in range(total_num_sector):
    assert ray.get(remote_objs[sector_id]) == 0

  2%|▏         | 3/173 [00:05<04:10,  1.47s/it]


In [16]:
2**30

1073741824

In [17]:
len(sector_filenames)

172

In [18]:
len(sector_file_descripts)

172

In [19]:
len(sector_filenames)

172

In [20]:
# write_data(sector_filenames, 
#            sector_file_descripts, 
#            os.path.join(sector_path, f"{sector_id}.jsonl"), 
#            0)